In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from os import makedirs
from os.path import join, exists, expanduser
from keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [2]:
img_des = pd.read_csv('D:\RMBI_MMKG_data\img_des.csv')
img_des

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,/m/05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,/m/024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,/m/02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."


In [3]:
img_des_trunc = img_des.head(100)
img_des_trunc

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
95,/m/02pprs,Alto saxophone,"The alto saxophone, also referred to as the al..."
96,/m/0d05q4,Iraq,"Iraq, officially the Republic of Iraq, is a co..."
97,/m/0kwgs,Madison County,Madison County is located in the north central...
98,/m/01gbbz,Ellen DeGeneres,"Ellen Lee DeGeneres is an American comedian, t..."


In [4]:
text_vec = pd.DataFrame(columns=['LabelName','name','title_vec','des_vec'])
text_vec['LabelName'] = img_des_trunc['LabelName']
text_vec['name'] = img_des_trunc['name']
text_vec['title_vec']=text_vec['title_vec'].astype(object)
text_vec['des_vec']=text_vec['des_vec'].astype(object)
text_vec

,LabelName,name,title_vec,des_vec
0,/m/027rn,Dominican Republic,NaN,NaN
1,/m/017dcd,Mighty Morphin Power Rangers,NaN,NaN
2,/m/01sl1q,Michelle Rodriguez,NaN,NaN
3,/m/0cnk2q,Australia national football team,NaN,NaN
4,/m/02_j1w,Defender,NaN,NaN
...,...,...,...,...
95,/m/02pprs,Alto saxophone,NaN,NaN
96,/m/0d05q4,Iraq,NaN,NaN
97,/m/0kwgs,Madison County,NaN,NaN
98,/m/01gbbz,Ellen DeGeneres,NaN,NaN


In [6]:
import sister
sentence_embedding = sister.BertEmbedding(lang="en")

for i in range(100):
    if (pd.notnull(img_des_trunc['name'][i])):
        text_vec['title_vec'][i] = sentence_embedding(img_des_trunc['name'][i])
    if (pd.notnull(img_des_trunc['description'][i])):
        text_vec['des_vec'][i] = sentence_embedding(img_des_trunc['description'][i]) 
text_vec

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,LabelName,name,title_vec,des_vec
0,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","[0.7342588, -0.70393395, 1.6656038, 0.67461014..."
1,/m/017dcd,Mighty Morphin Power Rangers,"[1.6969184, 1.8559471, 0.8122708, -0.40268016,...","[0.025732398, -0.7696621, 2.172137, 1.2750009,..."
2,/m/01sl1q,Michelle Rodriguez,"[1.2370641, 2.8018966, 0.16598952, -0.5138393,...","[0.6779196, -0.15636724, 0.9539355, 0.6223599,..."
3,/m/0cnk2q,Australia national football team,"[-0.34985536, 1.8396395, 1.8012114, -0.3399124...","[-0.7440083, 0.010402456, 2.7207577, 0.6726733..."
4,/m/02_j1w,Defender,"[0.10609877, 1.8506274, 1.07485, -0.12472837, ...","[-0.1949073, -0.38760465, 3.056767, 0.1185953,..."
...,...,...,...,...
95,/m/02pprs,Alto saxophone,"[1.2199156, 1.991265, 0.3569066, -0.048870564,...","[0.2933917, -0.43800604, 1.9724205, 1.4672562,..."
96,/m/0d05q4,Iraq,"[0.89850223, 2.5803485, -0.29931328, -0.114376...","[0.6169369, -0.20451215, 1.4815564, 1.3234156,..."
97,/m/0kwgs,Madison County,"[0.25071013, 1.1296625, 0.48001957, -0.4313162...","[-1.1063588, 0.3135531, 1.4151243, 0.30476758,..."
98,/m/01gbbz,Ellen DeGeneres,"[1.3937172, 2.8825302, -0.097079076, -1.545620...","[1.0277187, 0.039429538, -0.011159919, 0.72703..."


In [7]:
VGG_chopped = VGG16(weights='imagenet', include_top = True)
VGG_chopped = Model(inputs=VGG_chopped.inputs, outputs=VGG_chopped.layers[-2].output)

In [8]:
print('VGG_chopped.summary()',VGG_chopped.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
vec = pd.DataFrame(columns=['LabelName','name', 'title_vec','des_vec','img_vec'])
vec

,LabelName,name,title_vec,des_vec,img_vec


In [10]:
img_vec = pd.DataFrame(columns=['LabelName','name','vec'])
img_vec['vec']=img_vec['vec'].astype(object)
img_vec

,LabelName,name,vec


In [11]:
import os

for i in range(1):
    path_of_the_directory = os.path.join('D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader', img_des_trunc['name'][i])
    ext = ('.jpg')
    for path, dirc, files in os.walk(path_of_the_directory):
        for name in files:
            if name.endswith(ext):
                
                img_path = os.path.join(path, name)
                print(img_path)
                
                img = image.load_img(img_path, target_size=(224, 224))
                img = image.img_to_array(img)
                
                x = preprocess_input(np.expand_dims(img.copy(), axis=0))
                x = VGG_chopped.predict(x)
                
                vec_tmp = pd.DataFrame([[img_des_trunc['LabelName'][i], img_des_trunc['name'][i], text_vec['title_vec'][i], text_vec['des_vec'], x]], columns=['LabelName','name', 'title_vec','des_vec','img_vec'])                             
                vec = vec.append(vec_tmp, ignore_index=True)

D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\69784245.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\caribbean-beach-and-dominican-republ.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\corales.jpg


C:\Users\Alex\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\dominican+republic+(2).jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\dominican-republic-resort-cover.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\dominican-republic-tourism.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\DR-Heritage-Tour-February-2020-1-5.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\istock-1138303642.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dominican Republic\TAM_2187-ccb2af376cca452fbe4d84826e7.jpg


In [12]:
vec

,LabelName,name,title_vec,des_vec,img_vec
0,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[0.0, 1.0954936, 0.0, 0.0, 1.1145527, 0.0, 0...."
1,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[0.0, 2.649771, 1.1343346, 0.0, 0.0, 0.0, 0.0..."
2,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[2.3173857, 6.009555, 0.0, 0.0, 5.6593423, 0...."
3,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[0.0, 0.0, 0.0, 0.0, 1.1312225, 0.0, 0.0, 1.1..."
4,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[0.0, 0.0, 0.0, 0.0, 2.9732351, 0.0, 0.156601..."
5,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[0.0, 1.035866, 0.0, 0.0, 0.42709577, 0.0, 0...."
6,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[0.740768, 1.3790102, 0.0, 0.0, 2.8708286, 0...."
7,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[2.2376466, 1.3440793, 0.0, 0.0, 1.0441806, 0..."
8,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","0 [0.7342588, -0.70393395, 1.6656038, 0.67...","[[0.63865626, 3.0698574, 0.3681985, 0.0, 1.837..."


In [19]:
vec['title_vec'][0].shape

(768,)

In [20]:
vec['des_vec'][0].shape

(100,)

In [21]:
vec['img_vec'][0].shape

(1, 4096)

In [32]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

text_input = keras.Input(shape=(None,), name="text")
entity_title_input = keras.Input(shape=(None,), name="entity_title")
img_input = keras.Input(shape=(None,), name="img")

#embed into 64-dimentional
text_features = layers.Embedding(300, 64)(text_input)
entity_title_features = layers.Embedding(300, 64)(entity_title_input)
img_features = layers.Embedding(4096, 64)(img_input)

combinedInput = layers.concatenate([entity_title_features, text_features, img_features])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

model = Model(inputs=[text_input, entity_title_input, img_input], outputs=x)

In [33]:
combinedInput

<KerasTensor: shape=(None, None, 192) dtype=float32 (created by layer 'concatenate_7')>